In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [ ]:
train = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Train (12).csv")
test = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Test (13).csv")
sample = pd.read_csv('../input/landslide-prevention-and-innovation-challenge/SampleSubmission (7).csv')

In [ ]:
train["Label"].value_counts()

In [ ]:
train

In [ ]:
id_=test["Sample_ID"]
test.drop("Sample_ID", axis=1, inplace=True)
train.drop("Sample_ID", axis=1, inplace=True)

In [ ]:
col=train.columns[:-1]

In [ ]:
col

In [ ]:
train1=pd.DataFrame()
test1=pd.DataFrame()

In [ ]:
train["elevation_mean"]=train[train.columns[:25].to_list()].mean(axis=1)
train["slope_mean"]=train[train.columns[25:50].to_list()].mean(axis=1)
train["aspect_mean"]=train[train.columns[50:75].to_list()].mean(axis=1)
train["placurv_mean"]=train[train.columns[75:100].to_list()].mean(axis=1)
train["procurv_mean"]=train[train.columns[100:125].to_list()].mean(axis=1)
train["lsfactor_mean"]=train[train.columns[125:150].to_list()].mean(axis=1)
train["twi_mean"]=train[train.columns[150:175].to_list()].mean(axis=1)
train["geology_mean"]=train[train.columns[175:200].to_list()].mean(axis=1)
train["sdoif_mean"]=train[train.columns[200:225].to_list()].mean(axis=1)
########################################################################
train["elevation_count"]=train[train.columns[:25].to_list()].count(axis=1)
train["slope_count"]=train[train.columns[25:50].to_list()].count(axis=1)
train["aspect_count"]=train[train.columns[50:75].to_list()].count(axis=1)
train["placurv_count"]=train[train.columns[75:100].to_list()].count(axis=1)
train["procurv_count"]=train[train.columns[100:125].to_list()].count(axis=1)
train["lsfactor_count"]=train[train.columns[125:150].to_list()].count(axis=1)
train["twi_count"]=train[train.columns[150:175].to_list()].count(axis=1)
train["geology_count"]=train[train.columns[175:200].to_list()].count(axis=1)
train["sdoif_count"]=train[train.columns[200:225].to_list()].count(axis=1)
#########################################################################
train["elevation_std"]=train[train.columns[:25].to_list()].std(axis=1)
train["slope_std"]=train[train.columns[25:50].to_list()].std(axis=1)
train["aspect_std"]=train[train.columns[50:75].to_list()].std(axis=1)
train["placurv_std"]=train[train.columns[75:100].to_list()].std(axis=1)
train["procurv_std"]=train[train.columns[100:125].to_list()].std(axis=1)
train["lsfactor_std"]=train[train.columns[125:150].to_list()].std(axis=1)
train["twi_std"]=train[train.columns[150:175].to_list()].std(axis=1)
train["geology_std"]=train[train.columns[175:200].to_list()].std(axis=1)
train["sdoif_std"]=train[train.columns[200:225].to_list()].std(axis=1)
#########################################################################
train["elevation_min"]=train[train.columns[:25].to_list()].min(axis=1)
train["slope_min"]=train[train.columns[25:50].to_list()].min(axis=1)
train["aspect_min"]=train[train.columns[50:75].to_list()].min(axis=1)
train["placurv_min"]=train[train.columns[75:100].to_list()].min(axis=1)
train["procurv_min"]=train[train.columns[100:125].to_list()].min(axis=1)
train["lsfactor_min"]=train[train.columns[125:150].to_list()].min(axis=1)
train["twi_min"]=train[train.columns[150:175].to_list()].min(axis=1)
train["geology_min"]=train[train.columns[175:200].to_list()].min(axis=1)
train["sdoif_min"]=train[train.columns[200:225].to_list()].min(axis=1)
#########################################################################
train["elevation_max"]=train[train.columns[:25].to_list()].max(axis=1)
train["slope_max"]=train[train.columns[25:50].to_list()].max(axis=1)
train["aspect_max"]=train[train.columns[50:75].to_list()].max(axis=1)
train["placurv_max"]=train[train.columns[75:100].to_list()].max(axis=1)
train["procurv_max"]=train[train.columns[100:125].to_list()].max(axis=1)
train["lsfactor_max"]=train[train.columns[125:150].to_list()].max(axis=1)
train["twi_max"]=train[train.columns[150:175].to_list()].max(axis=1)
train["geology_max"]=train[train.columns[175:200].to_list()].max(axis=1)
train["sdoif_max"]=train[train.columns[200:225].to_list()].max(axis=1)
#########################################################################
train["elevation_kurt"]=train[train.columns[:25].to_list()].kurt(axis=1)
train["slope_kurt"]=train[train.columns[25:50].to_list()].kurt(axis=1)
train["aspect_kurt"]=train[train.columns[50:75].to_list()].kurt(axis=1)
train["placurv_kurt"]=train[train.columns[75:100].to_list()].kurt(axis=1)
train["procurv_kurt"]=train[train.columns[100:125].to_list()].kurt(axis=1)
train["lsfactor_kurt"]=train[train.columns[125:150].to_list()].kurt(axis=1)
train["twi_kurt"]=train[train.columns[150:175].to_list()].kurt(axis=1)
train["geology_kurt"]=train[train.columns[175:200].to_list()].kurt(axis=1)
train["sdoif_kurt"]=train[train.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
train["elevation_skew"]=train[train.columns[:25].to_list()].skew(axis=1)
train["slope_skew"]=train[train.columns[25:50].to_list()].skew(axis=1)
train["aspect_skew"]=train[train.columns[50:75].to_list()].skew(axis=1)
train["placurv_skew"]=train[train.columns[75:100].to_list()].skew(axis=1)
train["procurv_skew"]=train[train.columns[100:125].to_list()].skew(axis=1)
train["lsfactor_skew"]=train[train.columns[125:150].to_list()].skew(axis=1)
train["twi_skew"]=train[train.columns[150:175].to_list()].skew(axis=1)
train["geology_skew"]=train[train.columns[175:200].to_list()].skew(axis=1)
train["sdoif_skew"]=train[train.columns[200:225].to_list()].skew(axis=1)
#########################################################################
train["elevation_q7.5"]=train[train.columns[:25].to_list()].quantile(q=0.75,axis=1)
train["slope_q7.5"]=train[train.columns[25:50].to_list()].quantile(q=0.75,axis=1)
train["aspect_q7.5"]=train[train.columns[50:75].to_list()].quantile(q=0.75,axis=1)
train["placurv_q7.5"]=train[train.columns[75:100].to_list()].quantile(q=0.75,axis=1)
train["procurv_q7.5"]=train[train.columns[100:125].to_list()].quantile(q=0.75,axis=1)
train["lsfactor_q7.5"]=train[train.columns[125:150].to_list()].quantile(q=0.75,axis=1)
train["twi_q7.5"]=train[train.columns[150:175].to_list()].quantile(q=0.75,axis=1)
train["geology_q7.5"]=train[train.columns[175:200].to_list()].quantile(q=0.75,axis=1)
train["sdoif_q7.5"]=train[train.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
train["elevation_q5"]=train[train.columns[:25].to_list()].quantile(q=0.5,axis=1)
train["slope_q5"]=train[train.columns[25:50].to_list()].quantile(q=0.5,axis=1)
train["aspect_q5"]=train[train.columns[50:75].to_list()].quantile(q=0.5,axis=1)
train["placurv_q5"]=train[train.columns[75:100].to_list()].quantile(q=0.5,axis=1)
train["procurv_q5"]=train[train.columns[100:125].to_list()].quantile(q=0.5,axis=1)
train["lsfactor_q5"]=train[train.columns[125:150].to_list()].quantile(q=0.5,axis=1)
train["twi_q5"]=train[train.columns[150:175].to_list()].quantile(q=0.5,axis=1)
train["geology_q5"]=train[train.columns[175:200].to_list()].quantile(q=0.5,axis=1)
train["sdoif_q5"]=train[train.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
train["elevation_q2.5"]=train[train.columns[:25].to_list()].quantile(q=0.25,axis=1)
train["slope_q2.5"]=train[train.columns[25:50].to_list()].quantile(q=0.25,axis=1)
train["aspect_q2.5"]=train[train.columns[50:75].to_list()].quantile(q=0.25,axis=1)
train["placurv_q2.5"]=train[train.columns[75:100].to_list()].quantile(q=0.25,axis=1)
train["procurv_q2.5"]=train[train.columns[100:125].to_list()].quantile(q=0.25,axis=1)
train["lsfactor_q2.5"]=train[train.columns[125:150].to_list()].quantile(q=0.25,axis=1)
train["twi_q2.5"]=train[train.columns[150:175].to_list()].quantile(q=0.25,axis=1)
train["geology_q2.5"]=train[train.columns[175:200].to_list()].quantile(q=0.25,axis=1)
train["sdoif_q2.5"]=train[train.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
train.drop(col, axis=1, inplace=True)

In [ ]:

test["elevation_mean"]=test[test.columns[:25].to_list()].mean(axis=1)
test["slope_mean"]=test[test.columns[25:50].to_list()].mean(axis=1)
test["aspect_mean"]=test[test.columns[50:75].to_list()].mean(axis=1)
test["placurv_mean"]=test[test.columns[75:100].to_list()].mean(axis=1)
test["procurv_mean"]=test[test.columns[100:125].to_list()].mean(axis=1)
test["lsfactor_mean"]=test[test.columns[125:150].to_list()].mean(axis=1)
test["twi_mean"]=test[test.columns[150:175].to_list()].mean(axis=1)
test["geology_mean"]=test[test.columns[175:200].to_list()].mean(axis=1)
test["sdoif_mean"]=test[test.columns[200:225].to_list()].mean(axis=1)
########################################################################
test["elevation_count"]=test[test.columns[:25].to_list()].count(axis=1)
test["slope_count"]=test[test.columns[25:50].to_list()].count(axis=1)
test["aspect_count"]=test[test.columns[50:75].to_list()].count(axis=1)
test["placurv_count"]=test[test.columns[75:100].to_list()].count(axis=1)
test["procurv_count"]=test[test.columns[100:125].to_list()].count(axis=1)
test["lsfactor_count"]=test[test.columns[125:150].to_list()].count(axis=1)
test["twi_count"]=test[test.columns[150:175].to_list()].count(axis=1)
test["geology_count"]=test[test.columns[175:200].to_list()].count(axis=1)
test["sdoif_count"]=test[test.columns[200:225].to_list()].count(axis=1)
#########################################################################
test["elevation_std"]=test[test.columns[:25].to_list()].std(axis=1)
test["slope_std"]=test[test.columns[25:50].to_list()].std(axis=1)
test["aspect_std"]=test[test.columns[50:75].to_list()].std(axis=1)
test["placurv_std"]=test[test.columns[75:100].to_list()].std(axis=1)
test["procurv_std"]=test[test.columns[100:125].to_list()].std(axis=1)
test["lsfactor_std"]=test[test.columns[125:150].to_list()].std(axis=1)
test["twi_std"]=test[test.columns[150:175].to_list()].std(axis=1)
test["geology_std"]=test[test.columns[175:200].to_list()].std(axis=1)
test["sdoif_std"]=test[test.columns[200:225].to_list()].std(axis=1)
#########################################################################
test["elevation_min"]=test[test.columns[:25].to_list()].min(axis=1)
test["slope_min"]=test[test.columns[25:50].to_list()].min(axis=1)
test["aspect_min"]=test[test.columns[50:75].to_list()].min(axis=1)
test["placurv_min"]=test[test.columns[75:100].to_list()].min(axis=1)
test["procurv_min"]=test[test.columns[100:125].to_list()].min(axis=1)
test["lsfactor_min"]=test[test.columns[125:150].to_list()].min(axis=1)
test["twi_min"]=test[test.columns[150:175].to_list()].min(axis=1)
test["geology_min"]=test[test.columns[175:200].to_list()].min(axis=1)
test["sdoif_min"]=test[test.columns[200:225].to_list()].min(axis=1)
#########################################################################
test["elevation_max"]=test[test.columns[:25].to_list()].max(axis=1)
test["slope_max"]=test[test.columns[25:50].to_list()].max(axis=1)
test["aspect_max"]=test[test.columns[50:75].to_list()].max(axis=1)
test["placurv_max"]=test[test.columns[75:100].to_list()].max(axis=1)
test["procurv_max"]=test[test.columns[100:125].to_list()].max(axis=1)
test["lsfactor_max"]=test[test.columns[125:150].to_list()].max(axis=1)
test["twi_max"]=test[test.columns[150:175].to_list()].max(axis=1)
test["geology_max"]=test[test.columns[175:200].to_list()].max(axis=1)
test["sdoif_max"]=test[test.columns[200:225].to_list()].max(axis=1)
#########################################################################
test["elevation_kurt"]=test[test.columns[:25].to_list()].kurt(axis=1)
test["slope_kurt"]=test[test.columns[25:50].to_list()].kurt(axis=1)
test["aspect_kurt"]=test[test.columns[50:75].to_list()].kurt(axis=1)
test["placurv_kurt"]=test[test.columns[75:100].to_list()].kurt(axis=1)
test["procurv_kurt"]=test[test.columns[100:125].to_list()].kurt(axis=1)
test["lsfactor_kurt"]=test[test.columns[125:150].to_list()].kurt(axis=1)
test["twi_kurt"]=test[test.columns[150:175].to_list()].kurt(axis=1)
test["geology_kurt"]=test[test.columns[175:200].to_list()].kurt(axis=1)
test["sdoif_kurt"]=test[test.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
test["elevation_skew"]=test[test.columns[:25].to_list()].skew(axis=1)
test["slope_skew"]=test[test.columns[25:50].to_list()].skew(axis=1)
test["aspect_skew"]=test[test.columns[50:75].to_list()].skew(axis=1)
test["placurv_skew"]=test[test.columns[75:100].to_list()].skew(axis=1)
test["procurv_skew"]=test[test.columns[100:125].to_list()].skew(axis=1)
test["lsfactor_skew"]=test[test.columns[125:150].to_list()].skew(axis=1)
test["twi_skew"]=test[test.columns[150:175].to_list()].skew(axis=1)
test["geology_skew"]=test[test.columns[175:200].to_list()].skew(axis=1)
test["sdoif_skew"]=test[test.columns[200:225].to_list()].skew(axis=1)
#########################################################################
test["elevation_q7.5"]=test[test.columns[:25].to_list()].quantile(q=0.75,axis=1)
test["slope_q7.5"]=test[test.columns[25:50].to_list()].quantile(q=0.75,axis=1)
test["aspect_q7.5"]=test[test.columns[50:75].to_list()].quantile(q=0.75,axis=1)
test["placurv_q7.5"]=test[test.columns[75:100].to_list()].quantile(q=0.75,axis=1)
test["procurv_q7.5"]=test[test.columns[100:125].to_list()].quantile(q=0.75,axis=1)
test["lsfactor_q7.5"]=test[test.columns[125:150].to_list()].quantile(q=0.75,axis=1)
test["twi_q7.5"]=test[test.columns[150:175].to_list()].quantile(q=0.75,axis=1)
test["geology_q7.5"]=test[test.columns[175:200].to_list()].quantile(q=0.75,axis=1)
test["sdoif_q7.5"]=test[test.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
test["elevation_q5"]=test[test.columns[:25].to_list()].quantile(q=0.5,axis=1)
test["slope_q5"]=test[test.columns[25:50].to_list()].quantile(q=0.5,axis=1)
test["aspect_q5"]=test[test.columns[50:75].to_list()].quantile(q=0.5,axis=1)
test["placurv_q5"]=test[test.columns[75:100].to_list()].quantile(q=0.5,axis=1)
test["procurv_q5"]=test[test.columns[100:125].to_list()].quantile(q=0.5,axis=1)
test["lsfactor_q5"]=test[test.columns[125:150].to_list()].quantile(q=0.5,axis=1)
test["twi_q5"]=test[test.columns[150:175].to_list()].quantile(q=0.5,axis=1)
test["geology_q5"]=test[test.columns[175:200].to_list()].quantile(q=0.5,axis=1)
test["sdoif_q5"]=test[test.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
test["elevation_q2.5"]=test[test.columns[:25].to_list()].quantile(q=0.25,axis=1)
test["slope_q2.5"]=test[test.columns[25:50].to_list()].quantile(q=0.25,axis=1)
test["aspect_q2.5"]=test[test.columns[50:75].to_list()].quantile(q=0.25,axis=1)
test["placurv_q2.5"]=test[test.columns[75:100].to_list()].quantile(q=0.25,axis=1)
test["procurv_q2.5"]=test[test.columns[100:125].to_list()].quantile(q=0.25,axis=1)
test["lsfactor_q2.5"]=test[test.columns[125:150].to_list()].quantile(q=0.25,axis=1)
test["twi_q2.5"]=test[test.columns[150:175].to_list()].quantile(q=0.25,axis=1)
test["geology_q2.5"]=test[test.columns[175:200].to_list()].quantile(q=0.25,axis=1)
test["sdoif_q2.5"]=test[test.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
test.drop(col, axis=1, inplace=True)

In [ ]:
train.shape , test.shape

In [ ]:
y_train = train[['Label']]
df_train=train.drop(['Label'], axis=1)
cols=df_train.columns

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA()
# df_train = pca.fit_transform(df_train)
# test = pca.fit_transform(test)
# df_train = df_train[:, :15]
# test = test[:, :15]
# df_train = pd.DataFrame(df_train, columns=train.columns[:15])
# test = pd.DataFrame(test, columns=train.columns[:15])

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# sampler = RandomOverSampler(random_state=11)
# from imblearn.over_sampling import SMOTE
# sampler = SMOTE(k_neighbors=5, random_state=11)
from imblearn.over_sampling import BorderlineSMOTE
sampler = BorderlineSMOTE(k_neighbors=5, random_state=11)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
df_train = pd.DataFrame(ss.fit_transform(df_train), columns=df_train.columns)
test = pd.DataFrame(ss.transform(test), columns=test.columns)


In [ ]:
df_train.shape

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMClassifier(**{ 'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                                'n_estimators': 500,'colsample_bytree' : 0.8,
                                'seed': 42,'silent':False,
                                'early_stopping_rounds': 100,
                                'learning_rate' : 0.05,'num_iterations': 1500,
                                'max_depth' :8 ,'num_leaves' : 150,
                                'max_bins': 85,'min_data_in_leaf':30 })
cols=test.columns
test_ = test[cols]
Model_Name = "lgbm"
folds = StratifiedKFold(n_splits=14, shuffle=True, random_state=5168)
oofs  = np.zeros((len(df_train[cols])))
test_predictions = np.zeros((len(test)))
tst_predd = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train, train[['Label']])):
    X_train, y_train = df_train.iloc[trn_idx][cols], train.iloc[trn_idx]['Label']
    X_valid, y_valid = df_train.iloc[val_idx][cols], train.iloc[val_idx]['Label']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    X_train ,y_train = sampler.fit_resample(X_train ,y_train)
    
    clf = get_model(Name=Model_Name)
    clf.fit(X_train, y_train, eval_set = [(X_valid, y_valid)],
            verbose =100, early_stopping_rounds = 500)
    
    vp = clf.predict(X_valid)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_valid),squared=True)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = clf.predict(test_)
    test_predictions += tp / folds.n_splits
    tst_predd.append(tp)

  
print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(train["Label"], oofs,squared=True))
print("Model training")
# clf.fit(X_train, y_train )

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train["Label"], oofs))


           0       0.90      0.91      0.90      8148
           1       0.71      0.69      0.70      2716

    accuracy                           0.85     10864
   macro avg       0.80      0.80      0.80     10864
weighted avg       0.85      0.85      0.85     10864

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      8148
           1       0.73      0.70      0.72      2716

    accuracy                           0.86     10864
   macro avg       0.82      0.81      0.81     10864
weighted avg       0.86      0.86      0.86     10864µ

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      8148
           1       0.91      0.93      0.92      8148

    accuracy                           0.92     16296
   macro avg       0.92      0.92      0.92     16296
weighted avg       0.92      0.92      0.92     16296


           precision    recall  f1-score   support

           0       0.89      0.94      0.91      8148
           1       0.77      0.66      0.71      2716

    accuracy                           0.87     10864
   macro avg       0.83      0.80      0.81     10864
weighted avg       0.86      0.87      0.86     10864

In [ ]:
np.unique(test_predictions)

In [ ]:
for i in range(len(test_predictions)):
    if test_predictions[i]>=0.5:
        test_predictions[i]=1
    else:
        test_predictions[i]=0

In [ ]:
sub=pd.DataFrame({"Sample_ID":id_,"Label":test_predictions})

In [ ]:
sub.Label.value_counts()

In [ ]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

#### 